# Data Wranglig for Capstone Two Project 
## Project: Loan Default Prediction

LendingClub is a US peer-to-peer lending company. It was the first peer-to-peer lender to offer loan trading on a secondary market. LendingClub is the world's largest peer-to-peer lending platform which specializes in lending various types of loans to urban customers. When the company receives a loan application, the company has to make a decision for loan approval based on the applicant’s profile. Identifying the customers who are prone to default their loan ensures the company to enhance its financial standing. Failure in approving those who do not have the required qualifications and rejecting the applications who are eligible for the loans result in financial losses. Developing an accurate predictive model is required for the success of applicant’s screening.

## 1. Import packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import missingno as msno
#from library.sb_utils import save_file

In [2]:
#! pip install missingno

In [3]:
pwd

'C:\\Users\\saeed.rafieepour\\Desktop\\Training\\Springboard DS Course\\7. Data Wrangling-Capstone 2 Project\\7.6 Data Wrangling capstone project two\\notebooks'

### Useful Functions

In [4]:
# description of some columns
def feature_descriptor(feat_names=['loan_amnt','funded_amnt','funded_amnt_inv']):
    """ Returns the features description having the features names"""
    LCDataDictionary = pd.read_excel(r"C:\Users\saeed.rafieepour\Desktop\Training\Springboard DS Course\7. Data Wrangling-Capstone 2 Project\7.6 Data Wrangling capstone project two\raw data\archive\LCDataDictionary.xlsx", sheet_name=None)
    df_desc = LCDataDictionary['LoanStats']
    indexed_feature_desc = df_desc.set_index('LoanStatNew')
    return indexed_feature_desc.loc[feat_names,'Description'].values

In [5]:
# separating object variables vs numeric variables
def data_type(df):
    float64_dtype = df.select_dtypes(include=['float64'])
    int64_dtype = df.select_dtypes(include=['int64'])
    object_dtype = df.select_dtypes(include=['object'])
    return {'float64_dtype':float64_dtype,'int64_dtype':int64_dtype, 'object_dtype':object_dtype}

In [6]:
def category_to_numeric_convertor(df):
    object_dtype = df.select_dtypes(include=['object'])
    object_dtype_cat = object_dtype.apply(lambda col:col.astype('category'))
    object_dtype_categorized = object_dtype_cat.apply(lambda col:col.cat.codes)
    return object_dtype_categorized

## 2. Data Collection

In [7]:
loan_data = pd.read_csv("C:\\Users\\saeed.rafieepour\\Desktop\\Training\\Springboard DS Course\\7. Data Wrangling-Capstone 2 Project\\7.6 Data Wrangling capstone project two\\raw data\\archive\\loan.csv")

C:\Users\saeed.rafieepour\AppData\Local\Temp\ipykernel_16000\1586605488.py:1: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  loan_data = pd.read_csv("C:\\Users\\saeed.rafieepour\\Desktop\\Training\\Springboard DS Course\\7. Data Wrangling-Capstone 2 Project\\7.6 Data Wrangling capstone project two\\raw data\\archive\\loan.csv")


In [8]:
loan_data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,D1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,4000,4000,4000.0,36 months,18.94,146.51,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,30000,30000,30000.0,60 months,16.14,731.78,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260668 entries, 0 to 2260667
Columns: 145 entries, id to settlement_term
dtypes: float64(105), int64(4), object(36)
memory usage: 2.4+ GB


In [10]:
loan_data.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,url,dti,...,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
count,0.0,0.0,2.260668e+06,2.260668e+06,2.260668e+06,2.260668e+06,2.260668e+06,2.260664e+06,0.0,2.258957e+06,...,10613.0,10613.000000,10613.0,10613.000000,8426.000000,10613.000000,10613.000000,33056.000000,33056.000000,33056.000000
mean,NaN,NaN,1.504693e+04,1.504166e+04,1.502344e+04,1.309291e+01,4.458076e+02,7.799243e+04,NaN,1.882420e+01,...,3.0,155.006696,3.0,13.686422,454.840802,11628.036442,193.606331,5030.606922,47.775600,13.148596
std,NaN,NaN,9.190245e+03,9.188413e+03,9.192332e+03,4.832114e+00,2.671737e+02,1.126962e+05,NaN,1.418333e+01,...,0.0,129.113137,0.0,9.728138,375.830737,7615.161123,198.694368,3692.027842,7.336379,8.192319
min,NaN,NaN,5.000000e+02,5.000000e+02,0.000000e+00,5.310000e+00,4.930000e+00,0.000000e+00,NaN,-1.000000e+00,...,3.0,0.640000,3.0,0.000000,1.920000,55.730000,0.010000,44.210000,0.200000,0.000000
25%,NaN,NaN,8.000000e+03,8.000000e+03,8.000000e+03,9.490000e+00,2.516500e+02,4.600000e+04,NaN,1.189000e+01,...,3.0,59.370000,3.0,5.000000,174.967500,5628.730000,43.780000,2227.000000,45.000000,6.000000
50%,NaN,NaN,1.290000e+04,1.287500e+04,1.280000e+04,1.262000e+01,3.779900e+02,6.500000e+04,NaN,1.784000e+01,...,3.0,119.040000,3.0,15.000000,352.605000,10044.220000,132.890000,4172.855000,45.000000,14.000000
75%,NaN,NaN,2.000000e+04,2.000000e+04,2.000000e+04,1.599000e+01,5.933200e+02,9.300000e+04,NaN,2.449000e+01,...,3.0,213.260000,3.0,22.000000,622.792500,16114.940000,284.180000,6870.782500,50.000000,18.000000
max,NaN,NaN,4.000000e+04,4.000000e+04,4.000000e+04,3.099000e+01,1.719830e+03,1.100000e+08,NaN,9.990000e+02,...,3.0,943.940000,3.0,37.000000,2680.890000,40306.410000,1407.860000,33601.000000,521.350000,181.000000


In [11]:
#loan_data.columns
print(loan_data.columns.sort_values(ascending=True))

Index(['acc_now_delinq', 'acc_open_past_24mths', 'addr_state', 'all_util',
       'annual_inc', 'annual_inc_joint', 'application_type', 'avg_cur_bal',
       'bc_open_to_buy', 'bc_util',
       ...
       'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee',
       'total_rec_prncp', 'total_rev_hi_lim', 'url', 'verification_status',
       'verification_status_joint', 'zip_code'],
      dtype='object', length=145)


### Checking for data types

In [12]:
# data types
loan_data.dtypes

id                       float64
member_id                float64
loan_amnt                  int64
funded_amnt                int64
funded_amnt_inv          float64
                          ...   
settlement_status         object
settlement_date           object
settlement_amount        float64
settlement_percentage    float64
settlement_term          float64
Length: 145, dtype: object

In [13]:
# Showing all data types to distinguish categorial data types
loan_data.head(10)
pd.set_option('display.max_columns', loan_data.head().shape[1]+1)
print(loan_data.head())

   id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0 NaN        NaN       2500         2500           2500.0   36 months   
1 NaN        NaN      30000        30000          30000.0   60 months   
2 NaN        NaN       5000         5000           5000.0   36 months   
3 NaN        NaN       4000         4000           4000.0   36 months   
4 NaN        NaN      30000        30000          30000.0   60 months   

   int_rate  installment grade sub_grade       emp_title emp_length  \
0     13.56        84.92     C        C1            Chef  10+ years   
1     18.94       777.23     D        D2     Postmaster   10+ years   
2     17.97       180.69     D        D1  Administrative    6 years   
3     18.94       146.51     D        D2   IT Supervisor  10+ years   
4     16.14       731.78     C        C4        Mechanic  10+ years   

  home_ownership  annual_inc verification_status   issue_d loan_status  \
0           RENT     55000.0        Not Verified  Dec-2018  

In [14]:
# category data types:
'''
1. term 
2. grade
3. subgrade
4. emp_title 
5. emp_length
6. home_ownership  
7. verification_status   
8. loan_status  
9. pymnt_plan  
10. purpose
11. title
12. hardship_flag 
13. disbursement_method 
14. debt_settlement_flag  
15. zip_code
16. addr_state

'''

'\n1. term \n2. grade\n3. subgrade\n4. emp_title \n5. emp_length\n6. home_ownership  \n7. verification_status   \n8. loan_status  \n9. pymnt_plan  \n10. purpose\n11. title\n12. hardship_flag \n13. disbursement_method \n14. debt_settlement_flag  \n15. zip_code\n16. addr_state\n\n'

In [15]:
[data_type(loan_data)[i].shape for i in data_type(loan_data).keys()]

[(2260668, 105), (2260668, 4), (2260668, 36)]

In [16]:
data_type(loan_data)

{'float64_dtype':          id  member_id  funded_amnt_inv  int_rate  installment  annual_inc  \
 0       NaN        NaN           2500.0     13.56        84.92     55000.0   
 1       NaN        NaN          30000.0     18.94       777.23     90000.0   
 2       NaN        NaN           5000.0     17.97       180.69     59280.0   
 3       NaN        NaN           4000.0     18.94       146.51     92000.0   
 4       NaN        NaN          30000.0     16.14       731.78     57250.0   
 ...      ..        ...              ...       ...          ...         ...   
 2260663 NaN        NaN          12000.0     14.08       279.72     58000.0   
 2260664 NaN        NaN          12000.0     25.82       358.01     30000.0   
 2260665 NaN        NaN          10000.0     11.99       332.10     64000.0   
 2260666 NaN        NaN          12000.0     21.45       327.69     60000.0   
 2260667 NaN        NaN          16550.0     21.45       451.94     60000.0   
 
          url    dti  delinq_2yrs

### Number Of Missing Values By Column

In [17]:
def missing_calculator(df):
    missing_data = pd.concat([df.isna().sum(), 100 * df.isna().mean()], axis=1)
    missing_data.columns = ['count', 'missing %']
    return missing_data.sort_values(by='count', ascending=False)

In [18]:
missing_data = pd.concat([loan_data.isna().sum(), 100 * loan_data.isna().mean()], axis=1)
missing_data.columns = ['count', 'missing %']
missing_data.sort_values(by='count', ascending=False)

,count,missing %
id,2260668,100.000000
url,2260668,100.000000
member_id,2260668,100.000000
orig_projected_additional_accrued_interest,2252242,99.627278
hardship_length,2250055,99.530537
...,...,...
out_prncp_inv,0,0.000000
out_prncp,0,0.000000
initial_list_status,0,0.000000
hardship_flag,0,0.000000


In [19]:
#### just showing the columns where have missing values
only_missingcolumns = missing_data[missing_data['count']!=0].sort_values(by='count', ascending=False)
pd.set_option('display.max_rows', only_missingcolumns.shape[0]+1)
print(only_missingcolumns, '\n', f'The number of columns with missing values is: {only_missingcolumns.shape[0]}')

                                              count   missing %
id                                          2260668  100.000000
url                                         2260668  100.000000
member_id                                   2260668  100.000000
orig_projected_additional_accrued_interest  2252242   99.627278
hardship_length                             2250055   99.530537
hardship_reason                             2250055   99.530537
hardship_status                             2250055   99.530537
deferral_term                               2250055   99.530537
hardship_amount                             2250055   99.530537
hardship_start_date                         2250055   99.530537
hardship_end_date                           2250055   99.530537
payment_plan_start_date                     2250055   99.530537
hardship_loan_status                        2250055   99.530537
hardship_dpd                                2250055   99.530537
hardship_payoff_balance_amount          

### Strategy for removing null columns

In [20]:
## missing values less than percent%
percentage = 75
less_missing = missing_data[~(missing_data['missing %'] > percentage)] 
more_missing = missing_data[missing_data['missing %'] > percentage] 
print(f"{less_missing.sort_values(by='count', ascending=False)}\nNumber of columns with less than {percentage} % missing values is:{less_missing.shape[0]}")

                                  count  missing %
mths_since_last_major_derog     1679893  74.309585
mths_since_recent_revol_delinq  1520309  67.250432
next_pymnt_d                    1303607  57.664681
mths_since_last_delinq          1158502  51.246003
il_util                         1068850  47.280273
mths_since_rcnt_il               909924  40.250227
all_util                         866348  38.322655
open_acc_6m                      866130  38.313012
inq_last_12m                     866130  38.313012
total_cu_tl                      866130  38.313012
open_rv_24m                      866129  38.312968
open_il_12m                      866129  38.312968
open_il_24m                      866129  38.312968
total_bal_il                     866129  38.312968
open_rv_12m                      866129  38.312968
open_act_il                      866129  38.312968
inq_fi                           866129  38.312968
max_bal_bc                       866129  38.312968
mths_since_recent_inq          

In [21]:
# columns with more than percentage% missing values
print(more_missing.index, '\n', more_missing.shape[0])

Index(['id', 'member_id', 'url', 'desc', 'mths_since_last_record',
       'annual_inc_joint', 'dti_joint', 'verification_status_joint',
       'mths_since_recent_bc_dlq', 'revol_bal_joint',
       'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths',
       'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util',
       'sec_app_open_act_il', 'sec_app_num_rev_accts',
       'sec_app_chargeoff_within_12_mths',
       'sec_app_collections_12_mths_ex_med',
       'sec_app_mths_since_last_major_derog', 'hardship_type',
       'hardship_reason', 'hardship_status', 'deferral_term',
       'hardship_amount', 'hardship_start_date', 'hardship_end_date',
       'payment_plan_start_date', 'hardship_length', 'hardship_dpd',
       'hardship_loan_status', 'orig_projected_additional_accrued_interest',
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
       'settlement_amount', 'settlement_percent

In [22]:
## missing values more than percentage%
#less_missing = missing_data[~(missing_data['missing %'] > percentage)] 
#print(f"{less_missing.sort_index()}\nNumber of columns with missing values is:{less_missing.shape[0]}")

In [23]:
#LCDataDictionary = pd.read_excel(r"C:\Users\saeed.rafieepour\Desktop\Training\Springboard DS Course\7. Data Wrangling-Capstone 2 Project\7.6 Data Wrangling capstone project two\raw data\archive\LCDataDictionary.xlsx", sheet_name=None)
#set(less_missing.sort_index().index).difference(set(LCDataDictionary['browseNotes'].BrowseNotesFile))
#set(LCDataDictionary['browseNotes'].BrowseNotesFile) & set(less_missing.sort_index().index)

In [24]:
# filter columns with more than 75% of missing values
loan_data_less75perc_na = loan_data[less_missing.index.values]

In [25]:
print(loan_data_less75perc_na.shape)

(2260668, 105)


In [26]:
missing_calculator(loan_data_less75perc_na)

,count,missing %
mths_since_last_major_derog,1679893,74.309585
mths_since_recent_revol_delinq,1520309,67.250432
next_pymnt_d,1303607,57.664681
mths_since_last_delinq,1158502,51.246003
il_util,1068850,47.280273
mths_since_rcnt_il,909924,40.250227
all_util,866348,38.322655
open_acc_6m,866130,38.313012
inq_last_12m,866130,38.313012
total_cu_tl,866130,38.313012


In [27]:
loan_data_less75perc_na.mths_since_last_major_derog[~loan_data_less75perc_na.mths_since_last_major_derog.isna()]

8          45.0
10         22.0
13          6.0
15         48.0
16         68.0
           ... 
2260657    13.0
2260658    36.0
2260659    23.0
2260661    24.0
2260664    46.0
Name: mths_since_last_major_derog, Length: 580775, dtype: float64

In [28]:
feature_descriptor(['mths_since_last_major_derog', 'mths_since_recent_revol_delinq'])

array(['Months since most recent 90-day or worse rating',
       'Months since most recent revolving delinquency.'], dtype=object)

In [29]:
loan_data_less75perc_na.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,2500,2500,2500.0,36 months,13.56,84.92,C,C1,Chef,10+ years,RENT,55000.0,Not Verified,Dec-2018,Current,n,debt_consolidation,Debt consolidation,109xx,NY,18.24,0.0,Apr-2001,1.0,NaN,9.0,1.0,4341,10.3,34.0,w,2386.02,2386.02,167.02,167.02,113.98,53.04,0.0,0.0,0.0,Feb-2019,84.92,Mar-2019,Feb-2019,0.0,NaN,1,Individual,0.0,0.0,16901.0,2.0,2.0,1.0,2.0,2.0,12560.0,69.0,2.0,7.0,2137.0,28.0,42000.0,1.0,11.0,2.0,9.0,1878.0,34360.0,5.9,0.0,0.0,140.0,212.0,1.0,1.0,0.0,1.0,2.0,NaN,0.0,2.0,5.0,3.0,3.0,16.0,7.0,18.0,5.0,9.0,0.0,0.0,0.0,3.0,100.0,0.0,1.0,0.0,60124.0,16901.0,36500.0,18124.0,N,Cash,N
1,30000,30000,30000.0,60 months,18.94,777.23,D,D2,Postmaster,10+ years,MORTGAGE,90000.0,Source Verified,Dec-2018,Current,n,debt_consolidation,Debt consolidation,713xx,LA,26.52,0.0,Jun-1987,0.0,71.0,13.0,1.0,12315,24.2,44.0,w,29387.75,29387.75,1507.11,1507.11,612.25,894.86,0.0,0.0,0.0,Feb-2019,777.23,Mar-2019,Feb-2019,0.0,NaN,1,Individual,0.0,1208.0,321915.0,4.0,4.0,2.0,3.0,3.0,87153.0,88.0,4.0,5.0,998.0,57.0,50800.0,2.0,15.0,2.0,10.0,24763.0,13761.0,8.3,0.0,0.0,163.0,378.0,4.0,3.0,3.0,4.0,4.0,NaN,0.0,2.0,4.0,4.0,9.0,27.0,8.0,14.0,4.0,13.0,0.0,0.0,0.0,6.0,95.0,0.0,1.0,0.0,372872.0,99468.0,15000.0,94072.0,N,Cash,N
2,5000,5000,5000.0,36 months,17.97,180.69,D,D1,Administrative,6 years,MORTGAGE,59280.0,Source Verified,Dec-2018,Current,n,debt_consolidation,Debt consolidation,490xx,MI,10.51,0.0,Apr-2011,0.0,NaN,8.0,0.0,4599,19.1,13.0,w,4787.21,4787.21,353.89,353.89,212.79,141.10,0.0,0.0,0.0,Feb-2019,180.69,Mar-2019,Feb-2019,0.0,NaN,1,Individual,0.0,0.0,110299.0,0.0,1.0,0.0,2.0,14.0,7150.0,72.0,0.0,2.0,0.0,35.0,24100.0,1.0,5.0,0.0,4.0,18383.0,13800.0,0.0,0.0,0.0,87.0,92.0,15.0,14.0,2.0,77.0,14.0,NaN,0.0,0.0,3.0,3.0,3.0,4.0,6.0,7.0,3.0,8.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,136927.0,11749.0,13800.0,10000.0,N,Cash,N
3,4000,4000,4000.0,36 months,18.94,146.51,D,D2,IT Supervisor,10+ years,MORTGAGE,92000.0,Source Verified,Dec-2018,Current,n,debt_consolidation,Debt consolidation,985xx,WA,16.74,0.0,Feb-2006,0.0,NaN,10.0,0.0,5468,78.1,13.0,w,3831.93,3831.93,286.71,286.71,168.07,118.64,0.0,0.0,0.0,Feb-2019,146.51,Mar-2019,Feb-2019,0.0,NaN,1,Individual,0.0,686.0,305049.0,1.0,5.0,3.0,5.0,5.0,30683.0,68.0,0.0,0.0,3761.0,70.0,7000.0,2.0,4.0,3.0,5.0,30505.0,1239.0,75.2,0.0,0.0,62.0,154.0,64.0,5.0,3.0,64.0,5.0,NaN,0.0,1.0,2.0,1.0,2.0,7.0,2.0,3.0,2.0,10.0,0.0,0.0,0.0,3.0,100.0,100.0,0.0,0.0,385183.0,36151.0,5000.0,44984.0,N,Cash,N
4,30000,30000,30000.0,60 months,16.14,731.78,C,C4,Mechanic,10+ y

In [30]:
(loan_data_less75perc_na.loan_amnt==loan_data_less75perc_na.funded_amnt_inv).value_counts()

True     2109166
False     151502
dtype: int64

In [31]:
feature_descriptor(['loan_amnt','funded_amnt','funded_amnt_inv'])

array(['The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.',
       'The total amount committed to that loan at that point in time.',
       'The total amount committed by investors for that loan at that point in time.'],
      dtype=object)

In [32]:
# drop NA rows
loan_data_less75perc_na_ = loan_data_less75perc_na.dropna()

In [33]:
loan_data_less75perc_na_.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
8,5000,5000,5000.0,36 months,17.97,180.69,D,D1,Legal Assistant III,10+ years,MORTGAGE,53580.0,Source Verified,Dec-2018,Current,n,debt_consolidation,Debt consolidation,327xx,FL,21.16,0.0,Aug-1998,1.0,32.0,8.0,0.0,8018,35.2,38.0,w,4786.79,4786.79,353.89,353.89,213.21,140.68,0.0,0.0,0.0,Feb-2019,180.69,Mar-2019,Feb-2019,0.0,45.0,1,Individual,0.0,0.0,41882.0,5.0,2.0,5.0,5.0,3.0,33864.0,98.0,1.0,6.0,3132.0,73.0,22800.0,2.0,1.0,4.0,12.0,5235.0,13786.0,35.9,0.0,0.0,145.0,244.0,6.0,3.0,3.0,6.0,2.0,32.0,2.0,4.0,5.0,5.0,10.0,20.0,6.0,15.0,5.0,8.0,0.0,0.0,0.0,6.0,78.9,60.0,0.0,0.0,57426.0,41882.0,21500.0,34626.0,N,Cash,N
15,3500,3500,3500.0,36 months,16.14,123.30,C,C4,respritory therapist,10+ years,MORTGAGE,80000.0,Verified,Dec-2018,Current,n,debt_consolidation,Debt consolidation,604xx,IL,13.63,0.0,Dec-1988,0.0,43.0,7.0,0.0,8952,58.5,27.0,w,3346.52,3346.52,218.36,218.36,153.48,64.88,0.0,0.0,0.0,Feb-2019,123.30,Mar-2019,Feb-2019,0.0,48.0,1,Joint App,0.0,0.0,16085.0,2.0,2.0,0.0,0.0,57.0,7133.0,26.0,3.0,4.0,3873.0,38.0,15300.0,0.0,0.0,1.0,4.0,2298.0,4127.0,48.4,0.0,0.0,144.0,360.0,4.0,4.0,2.0,7.0,7.0,117.0,7.0,1.0,3.0,2.0,3.0,10.0,5.0,15.0,3.0,7.0,0.0,0.0,0.0,3.0,76.0,50.0,0.0,0.0,42328.0,16085.0,8000.0,27028.0,N,Cash,N
18,16000,16000,16000.0,60 months,20.89,431.87,D,D4,Neonatal Nurse Practitioner,4 years,MORTGAGE,120000.0,Not Verified,Dec-2018,Current,n,credit_card,Credit card refinancing,720xx,AR,27.57,1.0,Jul-2005,0.0,8.0,13.0,0.0,18826,82.2,37.0,w,15690.66,15690.66,835.89,835.89,309.34,526.55,0.0,0.0,0.0,Feb-2019,431.87,Mar-2019,Feb-2019,0.0,35.0,1,Individual,0.0,0.0,434239.0,1.0,6.0,1.0,4.0,4.0,247928.0,90.0,1.0,1.0,9439.0,87.0,22900.0,6.0,1.0,2.0,5.0,33403.0,430.0,96.8,0.0,0.0,136.0,146.0,12.0,4.0,2.0,12.0,4.0,8.0,2.0,2.0,4.0,2.0,5.0,24.0,6.0,10.0,4.0,13.0,0.0,0.0,0.0,2.0,86.5,100.0,0.0,0.0,444217.0,266754.0,13500.0,239338.0,N,Cash,N
23,3500,3500,3500.0,36 months,20.89,131.67,D,D4,gas attendant,10+ years,MORTGAGE,40000.0,Source Verified,Dec-2018,Current,n,car,Car financing,078xx,NJ,9.09,0.0,Oct-2004,1.0,24.0,4.0,0.0,1944,33.5,18.0,w,3357.29,3357.29,257.25,257.25,142.71,114.54,0.0,0.0,0.0,Feb-2019,131.67,Mar-2019,Feb-2019,0.0,48.0,1,Joint App,0.0,0.0,189794.0,1.0,1.0,1.0,1.0,7.0,24958.0,100.0,2.0,2.0,1317.0,87.0,5800.0,1.0,0.0,3.0,3.0,47449.0,3683.0,26.3,0.0,0.0,150.0,170.0,1.0,1.0,4.0,8.0,1.0,48.0,2.0,1.0,2.0,1.0,8.0,3.0,2.0,11.0,2.0,4.0,0.0,0.0,0.0,3.0,38.9,0.0,0.0,0.0,217000.0,26902.0,5000.0,25000.0,N,Cash,N
31,10000,10000,10000.0,60 months,19.92,264.

In [34]:
loan_data_less75perc_na_.shape

(98036, 105)

In [35]:
loan_data_less75perc_na_.describe().T

,count,mean,std,min,25%,50%,75%,max
loan_amnt,98036.0,14526.423967,9196.841835,1000.00,7500.0000,12000.000,20000.0000,40000.00
funded_amnt,98036.0,14526.423967,9196.841835,1000.00,7500.0000,12000.000,20000.0000,40000.00
funded_amnt_inv,98036.0,14521.661823,9195.417443,725.00,7500.0000,12000.000,20000.0000,40000.00
int_rate,98036.0,13.835739,4.877604,5.31,10.4700,13.490,16.1400,30.99
installment,98036.0,432.978771,267.406623,30.12,241.7000,361.040,571.2200,1714.54
annual_inc,98036.0,87138.443217,91638.190211,70.00,53000.0000,74000.000,102000.0000,8500000.00
dti,98036.0,19.016251,11.667435,0.00,12.4300,17.740,24.0700,999.00
delinq_2yrs,98036.0,0.768146,1.465929,0.00,0.0000,0.000,1.0000,35.00
inq_last_6mths,98036.0,0.577482,0.806818,0.00,0.0000,0.000,1.0000,5.00
mths_since_last_delinq,98036.0,33.713615,21.375022,0.00,16.0000,30.000,48.0000,161.00


### Further feature engineering

#### Exclude fully-paid loans 

In [36]:
# since we do not need to study those who have fully paied their loans, we only need to deal with those
# loan status that are charged off, late payments, etc. 
pd.options.display.max_rows = 1000
default_categories = ['Default', 'Charged Off', 'Does not meet the credit policy. Status:Charged Off']
loan_data_less75perc_na[loan_data_less75perc_na['loan_status'].isin(default_categories)].describe().T

,count,mean,std,min,25%,50%,75%,max
loan_amnt,262447.0,15531.548179,8797.625728,5.000000e+02,9000.000,14275.0000,20400.000000,4.000000e+04
funded_amnt,262447.0,15522.557031,8794.095373,5.000000e+02,9000.000,14225.0000,20300.000000,4.000000e+04
funded_amnt_inv,262447.0,15486.295030,8805.472861,0.000000e+00,8950.000,14100.0000,20202.031669,4.000000e+04
int_rate,262447.0,15.708974,4.893239,5.310000e+00,12.290,15.0500,18.550000,3.099000e+01
installment,262447.0,464.288595,262.355865,1.591000e+01,275.040,402.5400,603.850000,1.717630e+03
annual_inc,262447.0,70324.951694,65258.443710,0.000000e+00,43000.000,60000.0000,85000.000000,9.500000e+06
dti,262385.0,20.144915,11.286755,0.000000e+00,13.560,19.7600,26.300000,9.990000e+02
delinq_2yrs,262444.0,0.352330,0.939374,0.000000e+00,0.000,0.0000,0.000000,2.700000e+01
inq_last_6mths,262444.0,0.792577,1.050316,0.000000e+00,0.000,0.0000,1.000000,3.300000e+01
mths_since_last_delinq,134779.0,33.744337,21.992436,0.000000e+00,15.000,30.0000,49.000000,2.260000e+02


In [37]:
print('fico_range_high' in loan_data.columns)

False


In [38]:
# Converting zipcodes to only numbers by removing the xx from the end of them
loan_data_less75perc_na_.zip_code = loan_data_less75perc_na_.zip_code.str.replace("xx","")

C:\Users\saeed.rafieepour\AppData\Local\Temp\ipykernel_16000\3272680572.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loan_data_less75perc_na_.zip_code = loan_data_less75perc_na_.zip_code.str.replace("xx","")


In [39]:
loan_data_less75perc_na_.zip_code

8          327
15         604
18         720
23         078
31         648
          ... 
2260552    327
2260560    891
2260573    103
2260575    425
2260637    917
Name: zip_code, Length: 98036, dtype: object

### Converting categorial variables to numerical variables

In [40]:
object_dtype = data_type(loan_data_less75perc_na_)['object_dtype']

In [41]:
# Unique values of columns with object data type
# print([(i, object_dtype[i].unique()) for i in object_dtype.columns])
object_dtype.apply(lambda col: col.unique()).to_frame(name='Unique_Categories')

,Unique_Categories
term,"[ 36 months, 60 months]"
grade,"[D, C, E, B, A, F, G]"
sub_grade,"[D1, C4, D4, D3, E1, C3, B2, B4, B5, B1, C2, D..."
emp_title,"[Legal Assistant III, respritory therapist, Ne..."
emp_length,"[10+ years, 4 years, 5 years, 3 years, < 1 yea..."
home_ownership,"[MORTGAGE, RENT, OWN, ANY]"
verification_status,"[Source Verified, Verified, Not Verified]"
issue_d,"[Dec-2018, Nov-2018, Oct-2018, Sep-2018, Aug-2..."
loan_status,"[Current, In Grace Period, Late (16-30 days), ..."
pymnt_plan,"[n, y]"


In [42]:
object_dtype.apply(lambda col: col.nunique())

term                        2
grade                       7
sub_grade                  35
emp_title               36997
emp_length                 11
home_ownership              4
verification_status         3
issue_d                    37
loan_status                 5
pymnt_plan                  2
purpose                    12
title                      12
zip_code                  876
addr_state                 50
earliest_cr_line          621
initial_list_status         2
last_pymnt_d                6
next_pymnt_d                3
last_credit_pull_d         29
application_type            2
hardship_flag               2
disbursement_method         2
debt_settlement_flag        2
dtype: int64

In [43]:
#data feastures of object data type
feature_descriptor(['issue_d', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d'])

array(['The month which the loan was funded',
       'Last month payment was received', 'Next scheduled payment date',
       'The most recent month LC pulled credit for this loan'],
      dtype=object)

In [44]:
#loan_data_nona_categorized
object_dtype_categorized = category_to_numeric_convertor(loan_data_less75perc_na_)

In [45]:
object_dtype_categorized.head()

,term,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,earliest_cr_line,initial_list_status,last_pymnt_d,next_pymnt_d,last_credit_pull_d,application_type,hardship_flag,disbursement_method,debt_settlement_flag
8,0,3,15,16442,1,1,1,9,0,0,2,3,304,9,87,1,1,2,9,0,0,0,0
15,0,2,13,36081,1,1,2,9,0,0,2,3,533,13,126,1,1,2,9,1,0,0,0
18,1,3,18,19259,4,1,0,9,0,0,1,2,633,2,302,1,1,2,9,0,0,0,0
23,0,3,18,34577,1,1,1,9,0,0,0,1,67,30,560,1,1,2,9,1,0,0,0
31,1,3,17,18206,1,1,0,9,0,0,2,3,573,23,41,1,1,2,9,0,0,0,0


In [46]:
object_dtype_categorized.shape

(98036, 23)

In [47]:
# Removing the date columns
# df[df.columns.difference(['b'])]
# df.drop('b', axis=1)
# df.loc[:, df.columns != 'b']

#cat_col = object_dtype_categorized.loc[:,~object_dtype_categorized.columns.isin(['issue_d', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d'])]
#cat_col = object_dtype_categorized.drop(['issue_d', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d'], axis=1)
cat_col = object_dtype_categorized[object_dtype_categorized.columns.difference(['issue_d', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d'])]

In [48]:
cat_col.astype('category')

,addr_state,application_type,debt_settlement_flag,disbursement_method,earliest_cr_line,emp_length,emp_title,grade,hardship_flag,home_ownership,initial_list_status,loan_status,purpose,pymnt_plan,sub_grade,term,title,verification_status,zip_code
8,9,0,0,0,87,1,16442,3,0,1,1,0,2,0,15,0,3,1,304
15,13,1,0,0,126,1,36081,2,0,1,1,0,2,0,13,0,3,2,533
18,2,0,0,0,302,4,19259,3,0,1,1,0,1,0,18,1,2,0,633
23,30,1,0,0,560,1,34577,3,0,1,1,0,0,0,18,0,1,1,67
31,23,0,0,0,41,1,18206,3,0,1,1,0,2,0,17,1,3,0,573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260552,9,0,0,0,605,1,21809,1,0,1,1,0,3,0,6,0,6,2,304
2260560,32,0,0,0,561,3,19157,3,0,3,0,0,8,0,17,0,10,1,778
2260573,33,1,0,0,91,10,30649,3,0,3,0,0,2,0,16,0,3,0,88
2260575,16,0,0,0,288,1,26148,5,0,1,1,0,1,0,27,1,2,0,395


In [49]:
cat_col.dtypes

addr_state               int8
application_type         int8
debt_settlement_flag     int8
disbursement_method      int8
earliest_cr_line        int16
emp_length               int8
emp_title               int32
grade                    int8
hardship_flag            int8
home_ownership           int8
initial_list_status      int8
loan_status              int8
purpose                  int8
pymnt_plan               int8
sub_grade                int8
term                     int8
title                    int8
verification_status      int8
zip_code                int16
dtype: object

In [50]:
cat_col.shape

(98036, 19)

In [51]:
cat_col.columns
loan_data_less75perc_na_.loc[:,cat_col.columns].columns

Index(['addr_state', 'application_type', 'debt_settlement_flag',
       'disbursement_method', 'earliest_cr_line', 'emp_length', 'emp_title',
       'grade', 'hardship_flag', 'home_ownership', 'initial_list_status',
       'loan_status', 'purpose', 'pymnt_plan', 'sub_grade', 'term', 'title',
       'verification_status', 'zip_code'],
      dtype='object')

In [52]:
loan_data_less75perc_na_.loc[:,cat_col.columns]

,addr_state,application_type,debt_settlement_flag,disbursement_method,earliest_cr_line,emp_length,emp_title,grade,hardship_flag,home_ownership,initial_list_status,loan_status,purpose,pymnt_plan,sub_grade,term,title,verification_status,zip_code
8,FL,Individual,N,Cash,Aug-1998,10+ years,Legal Assistant III,D,N,MORTGAGE,w,Current,debt_consolidation,n,D1,36 months,Debt consolidation,Source Verified,327
15,IL,Joint App,N,Cash,Dec-1988,10+ years,respritory therapist,C,N,MORTGAGE,w,Current,debt_consolidation,n,C4,36 months,Debt consolidation,Verified,604
18,AR,Individual,N,Cash,Jul-2005,4 years,Neonatal Nurse Practitioner,D,N,MORTGAGE,w,Current,credit_card,n,D4,60 months,Credit card refinancing,Not Verified,720
23,NJ,Joint App,N,Cash,Oct-2004,10+ years,gas attendant,D,N,MORTGAGE,w,Current,car,n,D4,36 months,Car financing,Source Verified,078
31,MO,Individual,N,Cash,Apr-2005,10+ years,Material Handler,D,N,MORTGAGE,w,Current,debt_consolidation,n,D3,60 months,Debt consolidation,Not Verified,648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260552,FL,Individual,N,Cash,Sep-2000,10+ years,Police Officer,B,N,MORTGAGE,w,Current,home_improvement,n,B2,36 months,Home improvement,Verified,327
2260560,NV,Individual,N,Cash,Oct-2005,3 years,NURSE SUPERVISOR,D,N,RENT,f,Current,other,n,D3,36 months,Other,Source Verified,891
2260573,NY,Joint App,N,Cash,Aug-2002,< 1 year,Teacher,D,N,RENT,f,Current,debt_consolidation,n,D2,36 months,Debt consolidation,Not Verified,103
2260575,KY,Individual,N,Cash,Jul-1991,10+ years,Secretary,F,N,MORTGAGE,w,Current,credit_card,n,F3,60 months,Credit card refinancing,Not Verified,425


In [53]:
# Replace the columns in the cleaned dataframe with these categorized features
col = cat_col.columns.to_list()
loan_data_less75perc_nocat = loan_data_less75perc_na_.drop(labels=col, axis='columns')
#loan_data_less75perc_na_[col] = cat_col[col]

loan_data_less75perc_na_cat=pd.concat([loan_data_less75perc_nocat, cat_col], axis=1)

In [54]:
loan_data_less75perc_na_cat

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,issue_d,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,addr_state,application_type,debt_settlement_flag,disbursement_method,earliest_cr_line,emp_length,emp_title,grade,hardship_flag,home_ownership,initial_list_status,loan_status,purpose,pymnt_plan,sub_grade,term,title,verification_status,zip_code
8,5000,5000,5000.0,17.97,180.69,53580.0,Dec-2018,21.16,0.0,1.0,32.0,8.0,0.0,8018,35.2,38.0,4786.79,4786.79,353.89,353.89,213.21,140.68,0.0,0.0,0.0,Feb-2019,180.69,Mar-2019,Feb-2019,0.0,45.0,1,0.0,0.0,41882.0,5.0,2.0,5.0,5.0,3.0,33864.0,98.0,1.0,6.0,3132.0,73.0,22800.0,2.0,1.0,4.0,12.0,5235.0,13786.0,35.9,0.0,0.0,145.0,244.0,6.0,3.0,3.0,6.0,2.0,32.0,2.0,4.0,5.0,5.0,10.0,20.0,6.0,15.0,5.0,8.0,0.0,0.0,0.0,6.0,78.9,60.0,0.0,0.0,57426.0,41882.0,21500.0,34626.0,9,0,0,0,87,1,16442,3,0,1,1,0,2,0,15,0,3,1,304
15,3500,3500,3500.0,16.14,123.30,80000.0,Dec-2018,13.63,0.0,0.0,43.0,7.0,0.0,8952,58.5,27.0,3346.52,3346.52,218.36,218.36,153.48,64.88,0.0,0.0,0.0,Feb-2019,123.30,Mar-2019,Feb-2019,0.0,48.0,1,0.0,0.0,16085.0,2.0,2.0,0.0,0.0,57.0,7133.0,26.0,3.0,4.0,3873.0,38.0,15300.0,0.0,0.0,1.0,4.0,2298.0,4127.0,48.4,0.0,0.0,144.0,360.0,4.0,4.0,2.0,7.0,7.0,117.0,7.0,1.0,3.0,2.0,3.0,10.0,5.0,15.0,3.0,7.0,0.0,0.0,0.0,3.0,76.0,50.0,0.0,0.0,42328.0,16085.0,8000.0,27028.0,13,1,0,0,126,1,36081,2,0,1,1,0,2,0,13,0,3,2,533
18,16000,16000,16000.0,20.89,431.87,120000.0,Dec-2018,27.57,1.0,0.0,8.0,13.0,0.0,18826,82.2,37.0,15690.66,15690.66,835.89,835.89,309.34,526.55,0.0,0.0,0.0,Feb-2019,431.87,Mar-2019,Feb-2019,0.0,35.0,1,0.0,0.0,434239.0,1.0,6.0,1.0,4.0,4.0,247928.0,90.0,1.0,1.0,9439.0,87.0,22900.0,6.0,1.0,2.0,5.0,33403.0,430.0,96.8,0.0,0.0,136.0,146.0,12.0,4.0,2.0,12.0,4.0,8.0,2.0,2.0,4.0,2.0,5.0,24.0,6.0,10.0,4.0,13.0,0.0,0.0,0.0,2.0,86.5,100.0,0.0,0.0,444217.0,266754.0,13500.0,239338.0,2,0,0,0,302,4,19259,3,0,1,1,0,1,0,18,1,2,0,633
23,3500,3500,3500.0,20.89,131.67,40000.0,Dec-2018,9.09,0.0,1.0,24.0,4.0,0.0,1944,33.5,18.0,3357.29,3357.29,257.25,257.25,142.71,114.54,0.0,0.0,0.0,Feb-2019,131.67,Mar-2019,Feb-2019,0.0,48.0,1,0.0,0.0,189794.0,1.0,1.0,1.0,1.0,7.0,24958.0,100.0,2.0,2.0,1317.0,87.0,5800.0,1.0,0.0,3.0,3.0,47449.0,3683.0,26.3,0.0,0.0,150.0,170.0,1.0,1.0,4.0,8.0,1.0,48.0,2.0,1.0,2.0,1.0,8.0,3.0,2.0,11.0,2.0,4.0,0.0,0.0,0.0,3.0,38.9,0.0,0.0,0.0,217000.0,26902.0,5000.0,25000.0,30,1,0,0,560,1,34577,3,0,1,1,0,0,0,18,0,1,1,67
31,10000,10000,10000.0,19.92,264.50,80000.0,Dec-2018,20.67,0.0,0.0,37.0,11.0,0.0,13198,73.7,19.0,9801.36,9801.36,512.40,512.40,198.64,313.76,0.0,0.0,0.0,Feb-2019,264.50,Mar-2019,Feb-2019,0.0,37.0,1,0.0,116.0,110120.0,0.0,2.0,0.0,1.0,15.0,24408.0,70.0,1.0,1.0,6521.0,72.0,17900.0,0.0,0.0,1.0,2.0,10011.0,779.0,90.3,0.0,0.0,148.0,164.0,11.0,11.0,1.0,149.0,11.0,37.0,0.0,2.0,8.0,2.0,4.0,6.0,8.0,12.0,8.0,11.0,0.0,0.0,0.0,1.0,84.2,50.0,0.0,0.0,170230.0,37606

In [55]:
loan_data_less75perc_na_cat.dtypes

loan_amnt                           int64
funded_amnt                         int64
funded_amnt_inv                   float64
int_rate                          float64
installment                       float64
annual_inc                        float64
issue_d                            object
dti                               float64
delinq_2yrs                       float64
inq_last_6mths                    float64
mths_since_last_delinq            float64
open_acc                          float64
pub_rec                           float64
revol_bal                           int64
revol_util                        float64
total_acc                         float64
out_prncp                         float64
out_prncp_inv                     float64
total_pymnt                       float64
total_pymnt_inv                   float64
total_rec_prncp                   float64
total_rec_int                     float64
total_rec_late_fee                float64
recoveries                        

#### Check for duplication accross all columns

In [56]:
duplicates = loan_data_less75perc_na_cat.duplicated(keep=False)

In [57]:
loan_data_less75perc_na_cat[duplicates]

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,issue_d,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,addr_state,application_type,debt_settlement_flag,disbursement_method,earliest_cr_line,emp_length,emp_title,grade,hardship_flag,home_ownership,initial_list_status,loan_status,purpose,pymnt_plan,sub_grade,term,title,verification_status,zip_code


### Treating the dates

In [58]:
loan_data_less75perc_na_.issue_d

8          Dec-2018
15         Dec-2018
18         Dec-2018
23         Dec-2018
31         Dec-2018
             ...   
2260552    Oct-2017
2260560    Oct-2017
2260573    Oct-2017
2260575    Oct-2017
2260637    Oct-2017
Name: issue_d, Length: 98036, dtype: object